In [1]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\jillk\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [69]:
mcas = pd.read_csv("mcas_2015.csv", usecols = ["School Code", "Subject", "P+A %", "Student Included", "CPI", "SGP"])
parcc_ela = pd.read_csv("parcc_ela_2015.csv", usecols = ["Org Code", "Stud. Incl. #", "L4+5 %", "Trans. CPI Mean", "Trans. SGP Median"])
parcc_math = pd.read_csv("parcc_math_2015.csv", usecols = ["Org Code", "Stud. Incl. #", "L4+5 %", "Trans. CPI Mean", "Trans. SGP Median"])

In [70]:
parcc_ela = parcc_ela.rename(columns={"Stud. Incl. #": "Students (ELA)", "L4+5 %": "Perc Proficient (ELA)", "Trans. CPI Mean": "CPI (ELA)", "Trans. SGP Median": "SGP (ELA)"})
parcc_math = parcc_math.rename(columns={"Stud. Incl. #": "Students (Math)", "L4+5 %": "Perc Proficient (Math)", "Trans. CPI Mean": "CPI (Math)", "Trans. SGP Median": "SGP (Math)"})

In [71]:
parcc = pd.merge(parcc_ela, parcc_math, how= "inner", on='Org Code')

In [72]:
parcc["Test"] = "PARCC"

In [73]:
mcas.head()

,School Code,Subject,P+A %,Student Included,CPI,SGP
0,4450105,ENGLISH LANGUAGE ARTS,70.0,804,87.1,55.0
1,4450105,MATHEMATICS,55.0,804,78.9,49.0
2,4450105,SCIENCE AND TECH/ENG,59.0,324,83.6,NaN
3,10505,ENGLISH LANGUAGE ARTS,93.0,91,97.8,49.0
4,10505,MATHEMATICS,89.0,90,95.0,44.0


In [74]:
math_mask = mcas["Subject"] == "MATHEMATICS"
ela_mask = mcas["Subject"] == "ENGLISH LANGUAGE ARTS"

In [75]:
mcas_math = mcas[math_mask]
mcas_ela = mcas[ela_mask]

In [76]:
mcas_ela = mcas_ela.rename(columns={"School Code": "Org Code", "Student Included": "Students (ELA)", "P+A %": "Perc Proficient (ELA)", "CPI": "CPI (ELA)", "SGP": "SGP (ELA)"})
mcas_math = mcas_math.rename(columns={"School Code": "Org Code", "Student Included": "Students (Math)", "P+A %": "Perc Proficient (Math)", "CPI": "CPI (Math)", "SGP": "SGP (Math)"})

In [77]:
mcas_ela = mcas_ela.drop(['Subject'], axis=1)
mcas_math = mcas_math.drop(['Subject'], axis=1)

In [78]:
mcas = pd.merge(mcas_ela, mcas_math, how="left", on= "Org Code")

In [79]:
mcas["Test"] = "MCAS"

In [80]:
tests = pd.concat([parcc, mcas])

In [81]:
tests["Year"] = 2015

In [82]:
tests.head()

,Org Code,Perc Proficient (ELA),Students (ELA),CPI (ELA),SGP (ELA),Perc Proficient (Math),Students (Math),CPI (Math),SGP (Math),Test,Year
0,4120530,45.0,294,82.6,39,41.0,291.0,74.3,51,PARCC,2015
1,30025,69.0,216,88.3,67,56.0,216.0,84.0,59,PARCC,2015
2,30305,61.0,403,90.2,53,57.0,400.0,81.8,64,PARCC,2015
3,4090205,46.0,117,78.0,16,44.0,117.0,81.4,41,PARCC,2015
4,90310,78.0,536,95.2,43,68.0,532.0,88.1,39,PARCC,2015


In [83]:
tests = tests.set_index('Org Code')
tests.to_csv("tests_all.csv")